In [90]:
import pandas as pd

# Reading Dataset

In [91]:
df_Jan = pd.read_csv('Ridership_Jan2023.csv')
df_Jan.shape
df_Jan.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,01/01/2023 12:00:00 AM,subway,240,"7 Av (F,G)",Brooklyn,metrocard,Metrocard - Full Fare,46,1,40.666270,-73.98031,POINT (-73.98031 40.66627)
1,01/01/2023 12:00:00 AM,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,metrocard,Metrocard - Other,33,0,40.752290,-73.99339,POINT (-73.99339 40.75229)
2,01/01/2023 12:00:00 AM,subway,453,82 St-Jackson Hts (7),Queens,omny,OMNY - Full Fare,22,0,40.747658,-73.88370,POINT (-73.8837 40.747658)
3,01/01/2023 12:00:00 AM,subway,9,"57 St-7 Av (N,Q,R,W)",Manhattan,metrocard,Metrocard - Seniors & Disability,13,0,40.764664,-73.98066,POINT (-73.98066 40.764664)
4,01/01/2023 12:00:00 AM,subway,357,"Beverly Rd (2,5)",Brooklyn,metrocard,Metrocard - Full Fare,1,0,40.645100,-73.94896,POINT (-73.94896 40.6451)


In [92]:
df_Feb = pd.read_csv('Ridership_Feb2023.csv')
df_Feb.shape

(1942348, 12)

In [93]:
df_Mar = pd.read_csv('Ridership_Mar2023.csv')
df_Mar.shape

(218926, 12)

In [94]:
df_Apr = pd.read_csv('Ridership_Apr2023.csv')
df_Apr.shape

C:\Users\afrai\AppData\Local\Temp\ipykernel_29756\3108674545.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_Apr = pd.read_csv('Ridership_Apr2023.csv')


(2098133, 12)

In [95]:
df_May = pd.read_csv('Ridership_May2023.csv')
df_May.shape

(2192627, 12)

In [96]:
df_Jun = pd.read_csv('Ridership_Jun2023.csv')
df_Jun.shape

(2122984, 12)

In [97]:
df_Jul = pd.read_csv('Ridership_Jul2023.csv')
df_Jul.shape

(2125938, 12)

In [98]:
df_Aug = pd.read_csv('Ridership_Aug2023.csv')
df_Aug.shape

(2162687, 12)

In [99]:
df_Sep = pd.read_csv('Ridership_Sep2023.csv')
df_Sep.shape

(2087132, 12)

In [100]:
df_Oct = pd.read_csv('Ridership_Oct2023.csv')
df_Oct.shape

(2204295, 12)

In [101]:
df_Nov = pd.read_csv('Ridership_Nov2023.csv')
df_Nov.shape

(2128966, 12)

In [102]:
df_Dec = pd.read_csv('Ridership_Dec2023.csv')
df_Dec.shape

(2195693, 12)

# Ridership Data Grouping

In [182]:
# Fare mapping
fare_map = {
    'Metrocard - Full Fare': 2.90,
    'Metrocard - Fair Fare': 1.45,
    'Metrocard - Seniors & Disability': 1.45,
    'Metrocard - Unlimited 7-Day': 2.13,
    'Metrocard - Unlimited 30-Day': 1.65,
    'Metrocard - Students': 0.00,
    'Metrocard - Other': 2.00,
    'OMNY - Full Fare': 2.90,
    'OMNY - Seniors & Disability': 1.45,
    'OMNY - Other': 2.00,
}

month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Build a dictionary dynamically from your existing variables
df_dict = {month: globals()[f'df_{month}'] for month in month_list}

for month in month_list:
    df = df_dict[month]
    df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])
    df['month'] = df['transit_timestamp'].dt.month
    df['hour'] = df['transit_timestamp'].dt.hour
    df['netRidership'] = df['ridership']- df['transfers']
    df['fare_cost'] = df['fare_class_category'].map(fare_map)
    df['hourly_spending'] = df['netRidership'] * df['fare_cost']
    df_dict[month] = df  # update back into the dictionary

for month in month_list:
    globals()[f'df_{month}'] = df_dict[month]

df_dict = {month: globals()[f'df_{month}'] for month in month_list}

monthly_groups = []  # list to hold each grouped DataFrame

for i, month in enumerate(month_list, start=1):
    df = df_dict[month]

    # Filter for rush hour
    df_rush = df[df['hour'].isin([6, 7, 8, 9, 10])].copy()
    df_rush['month'] = i

    # Group core stats
    grouped = df_rush.groupby(['month', 'station_complex_id']).agg({
        'netRidership': 'sum',
        'hourly_spending': 'sum'
    }).reset_index()

    # Pivot to compute fare class distribution
    fare_dist = df_rush.pivot_table(
        index='station_complex_id',
        columns='fare_class_category',
        values='netRidership',
        aggfunc='sum',
        fill_value=0
    )

    # Normalize to get percentages
    fare_dist_pct = fare_dist.div(fare_dist.sum(axis=1), axis=0)
    fare_dist_pct.columns = [f"{col}_pct" for col in fare_dist_pct.columns]
    fare_dist_pct = fare_dist_pct.reset_index()  # <--- fix here
    fare_dist_pct['month'] = i


    # Merge into grouped data
    grouped = grouped.merge(fare_dist_pct, on=['station_complex_id', 'month'], how='left')

    monthly_groups.append(grouped)

df_all_months_grouped = pd.concat(monthly_groups, ignore_index=True)

total_rush_hour_rows = sum(df[df['hour'].isin([6, 7, 8, 9, 10])].shape[0] for df in df_dict.values())
total_rush_hour_rows

df_all_months_grouped = df_all_months_grouped.rename(columns = {'hourly_spending' : 'monthly_spending'})

df_all_months_grouped['avg_fare_per_ride'] = df_all_months_grouped['monthly_spending']/df_all_months_grouped['netRidership']

In [183]:
df_all_months_grouped

,month,station_complex_id,netRidership,monthly_spending,Metrocard - Fair Fare_pct,Metrocard - Full Fare_pct,Metrocard - Other_pct,Metrocard - Seniors & Disability_pct,Metrocard - Students_pct,Metrocard - Unlimited 30-Day_pct,Metrocard - Unlimited 7-Day_pct,OMNY - Full Fare_pct,OMNY - Other_pct,OMNY - Seniors & Disability_pct,avg_fare_per_ride
0,1,1,113180,270508.17,0.015391,0.229643,0.053870,0.024165,0.058084,0.144646,0.070852,0.403154,0.000000,0.000194,2.390070
1,1,10,43046,107326.70,0.016355,0.214468,0.037146,0.022162,0.026251,0.085908,0.173419,0.424035,0.000000,0.000256,2.493303
2,1,100,15496,37260.18,0.033234,0.147135,0.031750,0.040978,0.057757,0.104866,0.078149,0.505743,0.000000,0.000387,2.404503
3,1,101,64661,158612.81,0.032802,0.151529,0.046581,0.033482,0.049582,0.092993,0.062279,0.530057,0.000031,0.000665,2.452990
4,1,103,6324,14891.15,0.039532,0.158918,0.046964,0.030519,0.059298,0.104206,0.128874,0.431689,0.000000,0.000000,2.354704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131,12,97,72669,176536.97,0.044022,0.132643,0.036769,0.017793,0.063425,0.086584,0.071612,0.546698,0.000028,0.000427,2.429330
5132,12,98,29476,66969.94,0.066461,0.156975,0.060218,0.042679,0.078505,0.084950,0.103576,0.405279,0.000068,0.001289,2.272016
5133,12,99,20931,43880.16,0.082605,0.123358,0.047680,0.059577,0.076012,0.210597,0.086809,0.310544,0.000191,0.002628,2.096420
5134,12,TRAM1,13840,32341.39,0.021315,0.110621,0.035477,0.022616,0.006142,0.109899,0.397254,0.292197,0.000000,0.004480,2.336806


In [184]:
import pandas as pd

# Fare mapping
fare_map = {
    'Metrocard - Full Fare': 2.90,
    'Metrocard - Fair Fare': 1.45,
    'Metrocard - Seniors & Disability': 1.45,
    'Metrocard - Unlimited 7-Day': 2.13,
    'Metrocard - Unlimited 30-Day': 1.65,
    'Metrocard - Students': 0.00,
    'Metrocard - Other': 2.00,
    'OMNY - Full Fare': 2.90,
    'OMNY - Seniors & Disability': 1.45,
    'OMNY - Other': 2.00,
}

month_list = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Load and preprocess all months
df_all = []

for month in month_list:
    df = globals()[f'df_{month}'].copy()
    df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])
    df['hour'] = df['transit_timestamp'].dt.hour
    df['netRidership'] = df['ridership'] - df['transfers']
    df['fare_cost'] = df['fare_class_category'].map(fare_map)
    df['hourly_spending'] = df['netRidership'] * df['fare_cost']
    df_all.append(df)

df_all = pd.concat(df_all, ignore_index=True)

# Filter for rush hour only
df_rush = df_all[df_all['hour'].isin([6, 7, 8, 9, 10])].copy()

# Group by station to calculate total net ridership and total spending
grouped = df_rush.groupby('station_complex_id').agg({
    'netRidership': 'sum',
    'hourly_spending': 'sum'
}).reset_index()

# Rename for clarity
grouped = grouped.rename(columns={'hourly_spending': 'total_spending'})

# Calculate average fare per ride
grouped = grouped[grouped['netRidership'] > 0]  # Avoid divide-by-zero
grouped['avg_fare_per_ride'] = grouped['total_spending'] / grouped['netRidership']

# Calculate fare class distribution (% per station)
fare_dist = df_rush.pivot_table(
    index='station_complex_id',
    columns='fare_class_category',
    values='netRidership',
    aggfunc='sum',
    fill_value=0
)

fare_dist_pct = fare_dist.div(fare_dist.sum(axis=1), axis=0)
fare_dist_pct.columns = [f"{col}_pct" for col in fare_dist_pct.columns]
fare_dist_pct = fare_dist_pct.reset_index()

# Merge into final grouped dataset
df_final = grouped.merge(fare_dist_pct, on='station_complex_id', how='left')

In [185]:
df_final

,station_complex_id,netRidership,total_spending,avg_fare_per_ride,Metrocard - Fair Fare_pct,Metrocard - Full Fare_pct,Metrocard - Other_pct,Metrocard - Seniors & Disability_pct,Metrocard - Students_pct,Metrocard - Unlimited 30-Day_pct,Metrocard - Unlimited 7-Day_pct,OMNY - Full Fare_pct,OMNY - Other_pct,OMNY - Seniors & Disability_pct
0,1,1271659,3088173.45,2.428460,0.015479,0.198511,0.052030,0.026712,0.048286,0.134144,0.070172,0.453421,0.000020,0.001225
1,10,606571,1533035.78,2.527381,0.014135,0.187991,0.028043,0.019337,0.018549,0.067893,0.206639,0.456637,0.000087,0.000689
2,100,172148,421531.77,2.448659,0.029626,0.137324,0.034470,0.036225,0.044642,0.105316,0.081291,0.530276,0.000128,0.000703
3,101,704067,1748666.77,2.483665,0.036553,0.132672,0.044001,0.034659,0.040184,0.085647,0.063238,0.562199,0.000065,0.000781
4,103,85369,203469.10,2.383407,0.044372,0.138036,0.045825,0.026508,0.048530,0.090548,0.152983,0.452600,0.000023,0.000574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,97,902145,2207460.76,2.446902,0.042031,0.137460,0.039344,0.015666,0.050991,0.102203,0.075084,0.536814,0.000008,0.000399
424,98,347852,802095.73,2.305853,0.065203,0.172674,0.056883,0.038534,0.064729,0.094808,0.110423,0.395849,0.000029,0.000868
425,99,255560,551335.57,2.157363,0.070743,0.141184,0.044428,0.063183,0.062694,0.203334,0.090581,0.321940,0.000074,0.001839
426,TRAM1,136000,316630.61,2.328166,0.017022,0.280669,0.043676,0.029588,0.007324,0.125029,0.369684,0.125088,0.000044,0.001875


# Merge with Demographics

In [197]:
demo = pd.read_csv('StationComplex_Demographics.csv')
demo

,Complex_ID,Latitude,Longitude,MEAN_Median_Inc,SUM_Total_Popu,SUM_Male_Total,SUM_Female_Tot,SUM_White_Alon,SUM_Black_Alon,SUM_Hispanic_L,SUM_Non_Hispan
0,398,40.773620,-73.959874,134178.75000,47275,20936,26339,40094,372,44312,2963
1,399,40.768141,-73.963870,177047.88890,44931,19397,25534,36445,576,41252,3679
2,403,40.746081,-73.982076,155538.00000,48537,23030,25507,29953,2794,43228,5309
3,404,40.743070,-73.984264,183811.00000,53878,26223,27655,34329,3474,48862,5016
4,405,40.739864,-73.986599,176921.10000,60407,28990,31417,39826,4033,55036,5371
...,...,...,...,...,...,...,...,...,...,...,...
440,134,40.669367,-73.901975,33788.75000,35506,13784,21722,2712,21915,23172,12334
441,135,40.664038,-73.900571,39861.66667,36315,14929,21386,2161,24813,25553,10762
442,136,40.658733,-73.899232,56750.50000,26510,11339,15171,1341,19246,20713,5797
443,137,40.650573,-73.899485,78412.80000,13630,5757,7873,651,10440,11673,1957


In [212]:
df_final['station_complex_id'] = df_final['station_complex_id'].astype(str)
demo['Complex_ID'] = demo['Complex_ID'].astype(str)

df_merged = df_final.merge(
    demo,
    left_on='station_complex_id',
    right_on='Complex_ID',
    how='left'
)

In [213]:
total_cols = [
    'SUM_Male_Total', 'SUM_Female_Tot', 'SUM_White_Alon',
    'SUM_Black_Alon', 'SUM_Hispanic_L', 'SUM_Non_Hispan'
]

# Convert each column to percentage and rename
for col in total_cols:
    pct_col = col.replace('SUM_', '') + '_pct'
    df_merged[pct_col] = df_merged[col] / df_merged['SUM_Total_Popu']

# Drop raw total columns and SUM_Total_Popu
df_merged = df_merged.drop(columns=total_cols + ['SUM_Total_Popu'] + ['Complex_ID'])

In [214]:
df_merged

,station_complex_id,netRidership,total_spending,avg_fare_per_ride,Metrocard - Fair Fare_pct,Metrocard - Full Fare_pct,Metrocard - Other_pct,Metrocard - Seniors & Disability_pct,Metrocard - Students_pct,Metrocard - Unlimited 30-Day_pct,...,OMNY - Seniors & Disability_pct,Latitude,Longitude,MEAN_Median_Inc,Male_Total_pct,Female_Tot_pct,White_Alon_pct,Black_Alon_pct,Hispanic_L_pct,Non_Hispan_pct
0,1,1271659,3088173.45,2.428460,0.015479,0.198511,0.052030,0.026712,0.048286,0.134144,...,0.001225,40.775036,-73.912034,107187.30000,0.500531,0.499469,0.652126,0.030850,0.786273,0.213727
1,10,606571,1533035.78,2.527381,0.014135,0.187991,0.028043,0.019337,0.018549,0.067893,...,0.000689,40.759901,-73.984139,116506.00000,0.581989,0.418011,0.602526,0.067479,0.864149,0.135851
2,100,172148,421531.77,2.448659,0.029626,0.137324,0.034470,0.036225,0.044642,0.105316,...,0.000703,40.706870,-73.953431,58047.11111,0.495446,0.504554,0.630317,0.048921,0.725970,0.274030
3,101,704067,1748666.77,2.483665,0.036553,0.132672,0.044001,0.034659,0.040184,0.085647,...,0.000781,40.708359,-73.957757,96845.63636,0.490270,0.509730,0.654078,0.046692,0.733830,0.266170
4,103,85369,203469.10,2.383407,0.044372,0.138036,0.045825,0.026508,0.048530,0.090548,...,0.000574,40.720280,-73.993915,89467.00000,0.489717,0.510283,0.447210,0.045911,0.879574,0.120426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,97,902145,2207460.76,2.446902,0.042031,0.137460,0.039344,0.015666,0.050991,0.102203,...,0.000399,40.697207,-73.935657,74112.10000,0.476306,0.523694,0.254514,0.323519,0.595581,0.404419
424,98,347852,802095.73,2.305853,0.065203,0.172674,0.056883,0.038534,0.064729,0.094808,...,0.000868,40.700260,-73.941126,66438.84615,0.466380,0.533620,0.333818,0.235381,0.578025,0.421975
425,99,255560,551335.57,2.157363,0.070743,0.141184,0.044428,0.063183,0.062694,0.203334,...,0.001839,40.703869,-73.947408,64591.72727,0.497344,0.502656,0.557292,0.078202,0.670264,0.329736
426,TRAM1,136000,316630.61,2.328166,0.017022,0.280669,0.043676,0.029588,0.007324,0.125029,...,0.001875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:
df_merged.to_csv("Final_Ridership_Demographics.csv", index=False)